# Język Python - Wykład 4.

## Context Manager 

    with expression [as (targets)]:
        code block with context of "targets"

* Możliwość utworzenia dynamicznego (w runtime) kontekstu dla wykonania sekcji kodu
* Typowy przykład: czytanie z pliku, czytanie następuje w kontekście otwarcia i zamknięcia pliku
* Obiekt context managera musi implementować protokół: ``__enter__()`` i ``__exit__()``
* [PEP 343](http://www.python.org/dev/peps/pep-0343/)

In [ ]:
with open('L4.ipynb') as notebook:
    print notebook.read(50)

In [ ]:
notebook

In [ ]:
class MyContextManager(object):
    
    def __enter__(self):
        print "Context prepared"
    
    def __exit__(self, exc_type, exc_value, traceback):
        print "Context closed ", exc_type, exc_value, traceback

        
with MyContextManager():
    print "Hello"

In [ ]:
class MyContextManager(object):
    
    def __enter__(self):
        print "Context prepared"
    
    def __exit__(self, exc_type, exc_value, traceback):
        print "Context closed ", exc_type, exc_value, traceback

        
with MyContextManager():
    raise Exception()

In [ ]:
class MyContextManager(object):
    
    def __init__(self, custom="Context"):
        self.custom = custom
        
    def __enter__(self):
        print "%s prepared" % self.custom
    
    def __exit__(self, exc_type, exc_value, traceback):
        print "%s closed " % self.custom, exc_type, exc_value, traceback

        
with MyContextManager('Dog'):
    print "Hau"

### Context manager z użyciem dekoratora

In [ ]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print "<%s>" % name,
    yield
    print "</%s>" % name,

with tag("h1"):
    print "foo",

print

with tag("div"):
    print "foo",



## Metody specjalne (cd..)

### ``__repr__``

In [ ]:
class T:
    pass

[T(), T(), T()]

In [ ]:
class T:
    def __repr__(self):
        return 'T class'

print [T(), T(), T()]
print repr(T())

### ``__str__`` i ``__unicode__``

In [ ]:
class Car:
    def __init__(self, vendor, model, ccm):
        self.vendor = vendor
        self.model = model
        self.ccm = ccm
        
    def __str__(self):
        return '%s %s (%.1f L)' % (self.vendor, self.model, self.ccm / 1000.0)

print Car('Porsche', '911', 3600)

In [ ]:
class Car:
    def __init__(self, vendor, model, ccm):
        self.vendor = vendor
        self.model = model
        self.ccm = ccm
        
    def __str__(self):
        return '%s %s (%.1f L)' % (self.vendor, self.model, self.ccm / 1000.0)
    
    def __unicode__(self):
        return u'Unicode ' + unicode(str(self))

print unicode(Car('Porsche', '911', 3600))

### ``__hash__``

In [ ]:
class T:
    def __hash__(self):
        return hash("fake hash")

{T(): 1}
        

In [ ]:
hash("fake hash")

### ``__nonzero__``

In [ ]:
class T:
    def __nonzero__(self):
        print id(self)
        if id(self) / 10 % 2 :
            return True
        else:
            return False
print bool(T())        

### Emulacja typu kontenera (cd..)

In [ ]:
class T():
   
    c = [1, 2, 3]

    def __len__(self):
        return len(self.c)
    
    def __contains__(self, x):
        return x in self.c
    
    def __iter__(self):
        for x in self.c:
            yield x
    
t = T()
print len(t)
print 2 in t

In [ ]:
iter(t)

In [ ]:
list(iter(t))

## New-style class vs classic classes

In [ ]:
class T:
    pass
print type(T)
print type(T())

In [ ]:
class T(object):
    pass
print type(T)
print(T())

* obiekty "starych" klas pythona są realizowane przy użyciu jednego wbudowanego typu "instance"
* obiekty nowych klas, są ni mniej ni więcej definiowanym przez użytkownika nowym typem
* stare klasy występują tylko w Pythonie 2.x jako kompatybilność wsteczna. Python 3 nie posiada ich

In [ ]:
class MyClass(): # old-style

    def __init__(self, param): 
        print 'init', param 

    def __new__(cls, param):
        print cls.__name__, param
        return object.__new__(cls) 

MyClass(1)

In [ ]:
class MyClass(object): # new-style

    def __init__(self, param): 
        print 'init', param 

    def __new__(cls, param):
        print cls.__name__, param
        return object.__new__(cls) 

MyClass(1)

## Dziedziczenie, mixiny i MRO

In [ ]:
class Car(object):
    
    _TOP_SPEED = None
    
    @property
    def TOP_SPEED(self):
        if self._TOP_SPEED is not None:
            return self._TOP_SPEED
        else:
            raise NotImplementedError('Provide a top speed for a car')
            
    def accelerate(self):
        print '%s accelerated to %d' % (self.__class__.__name__, self.TOP_SPEED)
        
    _color = 'black'
    def color(self):
        print "All cars should be %s" % self._color
    
    def drive(self):
        self.horn()
        self.accelerate()
    
Car().accelerate()

In [ ]:
class Fiat(Car):
    _TOP_SPEED = 150

class AlfaRomeo(Car):
    _TOP_SPEED = 250
    
    def accelerate(self, *args, **kwargs):
        print 'WROOOOOM ',
        super(self.__class__, self).accelerate(*args, **kwargs)
    
Fiat().accelerate()
Fiat().color()
AlfaRomeo().accelerate()

In [ ]:
class MyCar(object):
    _color = 'red'
    def color(self):
        print "My %s is %s!!!" % (self.__class__.__name__, self._color)
        
class MyFiatWithAlfaRomeoEngine(MyCar, AlfaRomeo, Fiat):
    pass

MyFiatWithAlfaRomeoEngine().accelerate()
MyFiatWithAlfaRomeoEngine().color()


In [ ]:
MyFiatWithAlfaRomeoEngine.__mro__

### Mixin

In [ ]:
class HornMixin(object):
    def horn(self):
        print "Beeeep! Beeeep!"
        
class Fiat(HornMixin, Car):
    _TOP_SPEED = 150

class AlfaRomeo(HornMixin, Car):
    _TOP_SPEED = 250
    
Fiat().drive()
AlfaRomeo().drive()        

In [ ]:
class OnlyRedCarsCanSpeedingMixin(object):
    
    @property
    def TOP_SPEED(self):
        if self._color is 'red':
            return self._TOP_SPEED
        else:
            return 50

class MyAlfa(OnlyRedCarsCanSpeedingMixin, MyCar, AlfaRomeo):
    pass

class Fiat(OnlyRedCarsCanSpeedingMixin, Fiat):
    pass

MyAlfa().accelerate()
Fiat().accelerate()

In [ ]:
class Car(object):
    
    TOP_SPEED = 100
           
    def accelerate(self):
        print '%s accelerated to %d' % (
            self.__class__.__name__, self.TOP_SPEED)
    
    def horn(self):
        raise NotImplementedError('Implement how to horn!!!')
    
    def drive(self):
        self.horn()
        self.accelerate()
        
Car().drive()        

### MRO

In [ ]:
O = object
class X(O): pass
class Y(O): pass
class A(X,Y): pass
class B(Y,X): pass
class C(A,B): pass
C.__mro__

In [ ]:
O = object
class F(O): pass
class E(O): pass
class D(O): pass
class C(D,F): pass
class B(D,E): pass
class A(B,C): pass
print A.__mro__ 

## flightweight pattern = ``__new__()`` + weakref

* Fabryka singletonów
* Stosujemy wtedy kiedy mamy **wiele obiektów** danego typu ale **niewiele różnych wartości** tych obiektów
* Stan obiektu (jego wartość) powinna być możliwa do wyabstrahowania
* Współdzielenie obiektów w celu zaoszczędzenia zasobów
* Obiekty tworzymy za pomocą fabryki, nowy jeśli nie ma, stary jeśli możemy reużytkować
* Nieprawidłowa implementacja może prowadzić do wycieków pamieci

In [ ]:
values = ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')
suits = ('h', 'c', 'd', 's')
 
class Card:
    def __init__(self, value, suit):
        self.value, self.suit = value, suit

    def __repr__(self):
        return "<Card: %s%s>" % (self.value, self.suit)

    def __eq__(self, card):
        return self.value == card.value and self.suit == card.suit

    def __ne__(self, card):
       return not self.__eq__(card)

In [ ]:
c1 = Card('J', 'h')
c1

In [ ]:
c2 = Card('J', 'h')

In [ ]:
c1 == c2

In [ ]:
id(c1), id(c2)

In [ ]:
id(c1) == id(c2)

In [ ]:
import weakref

class FlightweightCard(object):
    _CardPool = weakref.WeakValueDictionary()

    def __new__(cls, value, suit):
        obj = FlightweightCard._CardPool.get(value + suit, None)
        if not obj:
            obj = object.__new__(cls)
            FlightweightCard._CardPool[value + suit] = obj
            obj.value, obj.suit = value, suit

        return obj

In [ ]:
c3 = FlightweightCard('J', 'h')
c4 = FlightweightCard('J', 'h')

In [ ]:
c3 == c4

In [ ]:
id(c3) == id(c4)

## Wyjątki są też klasami!

In [ ]:
5 + '5'

In [ ]:
try:
    5 + '5'
except TypeError:
    print 'no-no'

In [ ]:
try:
    5 + '5'
except TypeError, e:
    print type(e), e

In [ ]:
try:
    5 + '5'
except (TypeError, KeyError), e:
    print type(e), e

In [ ]:
try:
    dict()[3]
except TypeError, e:
    print type(e), e
except KeyError, e:
    print "it's only key error"

In [ ]:
class WrongAnswerError(Exception):
    pass

try:
    raise WrongAnswerError('wrong wrong wrong!')
except WrongAnswerError:
    print 'good good good'
finally:
    print 'bye!'

In [ ]:
class A(object):
    pass

raise A

In [ ]:
import sys, traceback

def lumberjack():
    bright_side_of_death()

def bright_side_of_death():
    return tuple()[0]

try:
    lumberjack()
except IndexError:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print "*** print_tb:"
    traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)
    print "*** print_exception:"
    traceback.print_exception(exc_type, exc_value, exc_traceback,
                              limit=2, file=sys.stdout)
    print "*** print_exc:"
    traceback.print_exc()
    print "*** format_exc, first and last line:"
    formatted_lines = traceback.format_exc().splitlines()
    print formatted_lines[0]
    print formatted_lines[-1]
    print "*** format_exception:"
    print repr(traceback.format_exception(exc_type, exc_value,
                                          exc_traceback))
    print "*** extract_tb:"
    print repr(traceback.extract_tb(exc_traceback))
    print "*** format_tb:"
    print repr(traceback.format_tb(exc_traceback))
    print "*** tb_lineno:", exc_traceback.tb_lineno

### Wyjątki i dekoratory

In [ ]:
class retry(object):
    
    def __init__(self, num):
        self.num = num
    
    def __call__(self, f):
        def wrapped():
            while True:
                try:
                    f()
                except Exception, e:
                    if self.num > 0:
                        print 'Retry', self.num
                        self.num -= 1
                    else:
                        raise e
        return wrapped

@retry(10)
def i_always_fail():
    print 'try'
    raise Exception('I always fail')
    
i_always_fail()